In [1]:
#1、导包
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn import svm            
from sklearn import model_selection
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import string

In [2]:
#2、数据采集
normal_payloads=pd.read_csv('dmzo_nomal.csv',nrows=10000,names=['payload'])
xss_payloads=pd.read_csv('xssed.csv',nrows=10000,names=['payload'])

In [3]:
#3、数据处理(改：换行标签也是用来构造xss payload的一种方法，所以不需要去除)
# def remove_line_break(payloads):  #去除换行符
#     temp_payloads=[]
#     for payload in payloads:
#         payload=payload.replace("<br/>","")
#         temp_payloads.append(payload)
#     return temp_payloads
# xss_payloads=remove_line_break(xss_payloads['payload'])
normal_payloads=pd.DataFrame(normal_payloads)
xss_payloads=pd.DataFrame(xss_payloads)
# xss_payloads.columns=['payload']
normal_payloads.insert(1,'xss',np.zeros(normal_payloads.shape[0]))
xss_payloads.insert(1,'xss',np.ones(xss_payloads.shape[0]))
payloads=pd.concat([xss_payloads,normal_payloads])
payloads.head(15)

,payload,xss
0,form.search_text=Dell%22%3E%3Cscript%3Ealert(/...,1.0
1,site=message&msg=<script>alert(1)</script>,1.0
2,Itemid=%22onmouseover=alert%28document.cookie%...,1.0
3,uilang=en%22%3E%3Cscript%3Ealert%28document.co...,1.0
4,msg=<ScRiPt>alert('LastRider-CyberBellona')</S...,1.0
5,"q=""><script>alert('Xss By Atm0n3r')</script>&s...",1.0
6,"q=/""><script>alert('Xss By Atm0n3r')</script>",1.0
7,taal=nl&zoektext=%3Cscript%3Ealert%28%22XSS%22...,1.0
8,d=%22%20onmouseover%3dalert%28document.c<br/>o...,1.0
9,buscar=+%2F%2F%22%3E%27%3E%3CSCRIPT%3Ea<br/>le...,1.0


In [4]:
#4、特征提取
#特征1：payload长度
def length_count(payload):
    return len(payload)
payloads['length']=payloads['payload'].apply(length_count)

#特征2：%个数
def percent_count(payload):
    return payload.count('%')
payloads['percent_num']=payloads['payload'].apply(percent_count)

# 特征3：是否包含大写字母
def is_capital(payload):
    temp_payload=str(payload).lower()
    if temp_payload==str(payload):
        return 0
    else:
        return 1
payloads['is_capital']=payloads['payload'].apply(is_capital)


'''
def has_sensitive_words(payload):
    result=0
    sensitive_words=['script','xss','java','alert','iframe','style','marquee','body']
    lower_payload=str(payload).lower()  #先转换为小写
    for word in sensitive_words:
        if word in lower_payload:
            result+=1
        else:
            result+=0
    if result!=0:
        return 1
    else:
        return 0
'''


'''
#特征4：是否含有敏感单词
def has_sensitive_words(payload):
    result=0
    sensitive_words=['script','xss','java','alert','iframe','style','marquee','body','onmouse','onerror','onload','eval','prompt']
    for word in sensitive_words:
        if re.search( word,str(payload),re.I):
            result+=1
        else:
            result+=0
    if result!=0:
        return 1
    else:
        return 0

payloads['sensitive_words']=payloads['payload'].apply(has_sensitive_words)
'''
#特征4：是否还有xss常用敏感标签
def has_sensitive_label(payload):
    result=0
    sensitive_labels=['script','iframe','style','marquee','body']
    for word in sensitive_labels:
        if re.search( word,str(payload),re.I):
            result+=1
        else:
            result+=0
    if result!=0:
        return 1
    else:
        return 0

payloads['sensitive_label']=payloads['payload'].apply(has_sensitive_label)

#特征5：是否含有事件触发
def has_events_trigger(payload):
    result=0
    events_trigger=['onmouse','onerror','onload','onclick','onfocus','onstart','onscroll','ontoggle','onblur']
    for word in events_trigger:
        if re.search( word,str(payload),re.I):
            result+=1
        else:
            result+=0
    if result!=0:
        return 1
    else:
        return 0

payloads['events_trigger']=payloads['payload'].apply(has_events_trigger)

#特征6：伪协议、字符拼接、编码等敏感字符
def has_sensitive_words(payload):
    result=0
    sensitive_words=['xss','java','alert','eval','prompt','top','string']
    for word in sensitive_words:
        if re.search( word,str(payload),re.I):
            result+=1
        else:
            result+=0
    if result!=0:
        return 1
    else:
        return 0

payloads['sensitive_words']=payloads['payload'].apply(has_sensitive_words)


#特征7：大小写字母转换频率
def upper_lower_transfer(payload):
    pattern1=re.compile(r'[a-z][A-Z]')
    pattern2=re.compile(r'[A-Z][a-z]')
    result1=pattern1.findall(str(payload))
    result2=pattern2.findall(str(payload))
    return len(result1)+len(result2)
payloads['upper_lower_transfer']=payloads['payload'].apply(upper_lower_transfer)

#特征8：标签是否闭合
def is_angle_brackets_closed(payload):
    left = 0
    right = 0
    for character in payload:
        if(character == '<'):
            left+=1
        elif(character == '>'):
            right+=1
    if (left==right): 
        return 1
    else:
        return 0
payloads['is_angle_brackets_closed']=payloads['payload'].apply(is_angle_brackets_closed)

#特征9：敏感字符个数
def sensitive_character_count(payload):
    num=0
    sensitive_list=["<",">","(",")","\'","\\","&","/"]
    for character in sensitive_list:
        num+=str(payload).lower().count(character)
    return num
payloads['sensitive_character']=payloads['payload'].apply(sensitive_character_count)
payloads.head(10)

,payload,xss,length,percent_num,is_capital,sensitive_label,events_trigger,sensitive_words,upper_lower_transfer,is_angle_brackets_closed,sensitive_character
0,form.search_text=Dell%22%3E%3Cscript%3Ealert(/...,1.0,115,6,1,1,0,1,4,1,9
1,site=message&msg=<script>alert(1)</script>,1.0,42,0,0,1,0,1,0,1,8
2,Itemid=%22onmouseover=alert%28document.cookie%...,1.0,58,5,1,0,1,1,1,1,0
3,uilang=en%22%3E%3Cscript%3Ealert%28document.co...,1.0,71,8,1,1,0,1,2,1,4
4,msg=<ScRiPt>alert('LastRider-CyberBellona')</S...,1.0,52,0,1,1,0,1,16,1,9
5,"q=""><script>alert('Xss By Atm0n3r')</script>&s...",1.0,62,0,1,1,0,1,4,0,11
6,"q=/""><script>alert('Xss By Atm0n3r')</script>",1.0,45,0,1,1,0,1,3,0,11
7,taal=nl&zoektext=%3Cscript%3Ealert%28%22XSS%22...,1.0,69,9,1,1,0,1,3,1,4
8,d=%22%20onmouseover%3dalert%28document.c<br/>o...,1.0,227,8,1,0,1,1,7,1,18
9,buscar=+%2F%2F%22%3E%27%3E%3CSCRIPT%3Ea<br/>le...,1.0,87,15,1,1,0,0,1,1,3


In [5]:
payloads.tail(10)

,payload,xss,length,percent_num,is_capital,sensitive_label,events_trigger,sensitive_words,upper_lower_transfer,is_angle_brackets_closed,sensitive_character
9990,search_for=works&amp;search_by=com.composer_id...,0.0,115,0,0,0,0,0,0,1,4
9991,name=donalderb,0.0,14,0,0,0,0,0,0,1,0
9992,page=composers/composer.html&amp;id=32,0.0,38,0,0,0,0,0,0,1,2
9993,fuseaction=composer.fa_dsp_biography&amp;authp...,0.0,67,0,0,0,0,0,0,1,2
9994,num=52,0.0,6,0,0,0,0,0,0,1,0
9995,fuseaction=composer.fa_dsp_biography&amp;authp...,0.0,68,0,0,0,0,0,0,1,2
9996,fuseaction=composer.fa_dsp_biography&amp;authp...,0.0,68,0,0,0,0,0,0,1,2
9997,fuseaction=composer.fa_dsp_biography&amp;authp...,0.0,67,0,0,0,0,0,0,1,2
9998,url=pierre-even,0.0,15,0,0,0,0,0,0,1,0
9999,id=502,0.0,6,0,0,0,0,0,0,1,0


In [6]:
#5、数据分割
z,y,x=np.split(payloads,
             (1,2,),
             axis=1)
#print(x)
x_train,x_test,y_train,y_test=model_selection.train_test_split(x,              #所要划分的样本特征集
                                                               y,              #所要划分的样本结果
                                                               random_state=1, #随机数种子
                                                               test_size=0.3)  #测试样本占比
#print(y_test)

In [7]:
#6、特征评分
from sklearn.feature_selection import SelectKBest
from matplotlib import pyplot as plt
 
def plot_feature_scores(x, y, names=None):
 
    if not names:
        names = range(len(x[0]))
 
 
    # 使用 sklearn.feature_selection.SelectKBest 给特征打分
    slct = SelectKBest(k="all")
    slct.fit(x, y)
    scores = slct.scores_
 
    # 将特征按分数 从大到小 排序
    named_scores = zip(names, scores)
    sorted_named_scores = sorted(named_scores, key=lambda z: z[1], reverse=True)
 
    sorted_scores = [each[1] for each in sorted_named_scores]
    sorted_names = [each[0] for each in sorted_named_scores]
 
    y_pos = np.arange(len(names))           # 从上而下的绘图顺序
 
    # 绘图
    fig, ax = plt.subplots()
    ax.barh(y_pos, sorted_scores, height=0.7, align='center', color='#AAAAAA', tick_label=sorted_names)
    ax.set_yticks(y_pos)
    ax.set_xlabel('Feature Score')
    ax.set_ylabel('Feature Name')
    ax.invert_yaxis()
    ax.set_title('F_classif scores of the features.')
 
    # 4. 添加每个names的数字标签
    for score, pos in zip(sorted_scores, y_pos):
        ax.text(score + 20, pos, '%.1f' % score, ha='center', va='bottom', fontsize=8)
 
    plt.show()
names=['length','percent_num','is_capital','sensitive_label','events_trigger','sensitive_words','upper_lower_transfer','is_angle_brackets_closed','sensitive_character']
plot_feature_scores(x,y,names)
#若报错，将y改为y.values.astype('int').ravel()

D:\ApplicationsForLearning\Pycharm\Anaconda\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


<Figure size 640x480 with 1 Axes>

In [8]:
#7、模型搭建
#SVM分类器构建
def SVM_classifier():
    #clf = svm.SVC(C=0.8,kernel='rbf', gamma=50,decision_function_shape='ovr')
    clf = svm.SVC(C=0.5,                         #误差项惩罚系数,默认值是1
                  kernel='linear',               #线性核 kenrel="rbf":高斯核
                  decision_function_shape='ovr') #决策函数
    return clf
#训练模型   
def train(clf,x_train,y_train):
    clf.fit(x_train,         #训练集特征向量
            y_train.values.astype('int').ravel()) #训练集目标值

In [9]:
#8、模型评估
def print_accuracy(clf,x_test,y_test):
    accscore = clf.score(x_test, y_test.values.astype('int').ravel())
    print("精确率："+str(accscore))
def print_recall(clf,x_test,y_test):
    y_true = y_test  # 实际上的标签
    y_predict = clf.predict(x_test)  # 模型根据特征预测的标签
    print("召回率："+str(recall_score(y_true.values.astype('int').ravel(), y_predict, average='macro')))

#SVM模型评估
SVM_clf = SVM_classifier()
train(SVM_clf,x_train,y_train)
print('SVM模型评估：\n')
print_accuracy(SVM_clf,x_test,y_test)
print_recall(SVM_clf,x_test,y_test)

#随机森林
RF_clf = RandomForestClassifier()
train(RF_clf,x_train,y_train)
print('\n随机森林模型评估：\n')
print_accuracy(RF_clf,x_test,y_test)
print_recall(RF_clf,x_test,y_test)

SVM模型评估：

精确率：0.9825
召回率：0.9825868296749096

随机森林模型评估：

精确率：0.9871666666666666
召回率：0.9873270352875799


D:\ApplicationsForLearning\Pycharm\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [10]:
#直接使用classification_report进行模型评估
from sklearn.metrics import classification_report
y_true=y_test.values.astype('int').ravel()
y_pred=SVM_clf.predict(x_test)
print(classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98      2950
           1       0.99      0.98      0.98      3050

    accuracy                           0.98      6000
   macro avg       0.98      0.98      0.98      6000
weighted avg       0.98      0.98      0.98      6000



In [15]:
#9、模型运用
print('Please input the correct payload format.')
test_payload= input("input:")
test = [[length_count(test_payload),percent_count(test_payload),is_capital(test_payload),has_sensitive_label(test_payload),has_events_trigger(test_payload),has_sensitive_words(test_payload),upper_lower_transfer(test_payload),is_angle_brackets_closed(test_payload),sensitive_character_count(test_payload)]]
test_data= pd.DataFrame(test, columns=['length','percent_num','is_capital','sensitive_label','events_trigger','sensitive_words','upper_lower_transfer','is_angle_brackets_closed','sensitive_character'])
print(test_data)

Please input the correct payload format.
input:ring=index
   length  percent_num  is_capital  sensitive_label  events_trigger  \
0      10            0           0                0               0   

   sensitive_words  upper_lower_transfer  is_angle_brackets_closed  \
0                0                     0                         1   

   sensitive_character  
0                    0  


In [16]:
#SVM模型预测
SVM_predict = SVM_clf.predict(test_data)
print('SVM模型预测：\n')
if(SVM_predict == 1):
    print("xss payload!")
else:
    print("normal payload!")
    
#随机森林模型预测
RF_predict = RF_clf.predict(test_data)
print('\nRF模型预测：\n')
if(RF_predict == 1):
    print("xss payload!")
else:
    print("normal payload!")

SVM模型预测：

normal payload!

RF模型预测：

normal payload!
